## Workbook of modal harmony (Palestrina counterpoint)
#### Analysis of G.P.da Palestrina Agnus from music21

In [1]:
from pcsPy import *
import pickle, copy
from IPython.display import Image 
sys.path.append('../')
from tonalHarmonyDefs import enharmonicDictionary, shortHands, scoreAnalysis, tonalPartition, showAnalysis

<IPython.core.display.Javascript object>

In [2]:
# Display width settings
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Here we import the tonal model we are going to use for the analysis. A tonal model is the matrix of all the operators that connect the chords of a chord library as roman numerals.
Typical models are built using combinations of basic chord symbols and possible extension. 

Use tonalHarmonyModel.py to generate or modify models

In [3]:
score = '/Users/marco/Dropbox (Personal)/Musica/Scores/Palestrina/Papae_Marcelli_Kyrie_6.mid.xml'
seq,chords = readScore(score,music21=False,show=False)

In [ ]:
score = m21.corpus.parse('palestrina/Agnus_II_00.krn')
score.show()

In [ ]:
score.show()

In [ ]:
# To write chords and corresponding operators - uses musescore
WRITEscoreOps(seq,w=True)

In [4]:
dnodes,dedges,dcounts,deg,modul,Gx,Gxu = scoreNetwork(seq,ntx=True,general=True)
print('average degree = ',deg,' modularity = ',modul)

average degree =  2.7096774193548385  modularity =  0.3376946366782007


In [6]:
# Write to csv for visualization in Gephi or other network drawing software (see figure below)
dnodes.to_csv('nodes_kyrie.csv')
dedges.to_csv('edges_kyrie.csv')

## ![title](Agnus.png)

In [5]:
moduldict,modul = tonalPartition(seq,chords,dnodes,Gx,Gxu,resolution=1.0,display=True)
modul.to_csv('~/Desktop/modul.csv')

In [ ]:
dendro1 = cm.generate_dendrogram(Gxu,part_init=part2)

In [ ]:
part2 = cm.best_partition(Gxu)

In [ ]:
for level in range(len(dendro1)) :
    print("partition at level", level,"is", cm.partition_at_level(dendro1, level))

In [ ]:
# f = open('moduldict.pkl','wb')
# pickle.dump(moduldict,f)
# f.close()

In [ ]:
# f = open('moduldict.pkl','rb')
# moduldict = pickle.load(f)
# f.close()
# modul = pd.read_csv('modulPalestrina.csv')

In [ ]:
modul[['Label','Degree','Modularity']].query('Modularity == 5').sort_values(by=['Degree'],ascending=False)

In [8]:
key = 'C'
rn = []
seqch = []
for d in dnodes.loc[0,'Label']:
    chstring = []
    for i in range(len(d)):
        if d[i] == '-' or d[i] == '#':
            chstring.pop()
            chstring.append(d[i-1]+d[i])
        else:
            chstring.append(d[i])
    n = m21.chord.Chord(chstring)
    rnum = m21.roman.romanNumeralFromChord(n,m21.key.Key(key)).romanNumeralAlone
    fig = m21.roman.postFigureFromChordAndKey(n,m21.key.Key(key))
    rn.append(rnum+fig)
    seqch.append(n.pitchClasses)

In [ ]:
# make table of operators
opstable = []
for n in range(len(seqch)):
    mat = []
    for m in range(len(seqch)):
        _,m = generalizedOpsName(seqch[n],seqch[m])
        mat.append(m)
    opstable.append(mat)

In [85]:
summary = []
for n in range(len(seqch)):
    summary.append([seqch[n],rn[n],PCSet(m21.roman.RomanNumeral(rn[n],key).pitchClasses).normalOrder().tolist()])
summary = pd.DataFrame(summary,columns=['Initial','RN','Final'])

In [10]:
summary.to_csv('summary.csv')

In [86]:
summary_old = pd.read_csv('palestrina.modal.C.csv',usecols=['Initial','RN','Final'])

In [87]:
summary_new = pd.concat([summary_old,summary], keys=['Initial','RN','Final'])

In [88]:
len(summary_new)

118

In [89]:
summary_cleaned = summary_new.loc[summary_new.astype(str).drop_duplicates(subset='Initial').index]

In [106]:
len(summary_cleaned)

78

In [91]:
summary_cleaned.to_csv('summary_clean.csv')

In [107]:
summary = pd.read_csv('palestrina.modal.Cext.csv',usecols=['Initial','RN','Final'])

In [108]:
summary

,Initial,RN,Final
0,"[9, 11, 0, 4]",vi763,"[9, 11, 0, 4]"
1,"[9, 11, 2]",vii65[no5],"[6, 9, 11, 2]"
2,"[9, 11, 2, 5]",vii/o65,"[9, 11, 2, 5]"
3,"[9, 1, 4]",VI,"[9, 1, 4]"
4,"[9, 0, 2]",ii62,"[9, 0, 2]"
...,...,...,...
73,"[5, 9, 11]",vii/o7[no3],"[9, 11, 2, 5]"
74,"[5, 7, 9, 11, 0]",IV7654,"[5, 7, 9, 11, 0]"
75,"[7, 9, 10]",vb32,"[7, 9, 10]"
76,"[7, 10, 11]",v3b3,"[7, 10]"


In [109]:
rn = summary['RN'].tolist()
seqch = []
for n in summary['Initial'].tolist():
    seqch.append(list(eval(n)))

In [111]:
# make table of operators
opstable = []
for n in range(len(seqch)):
    mat = []
    for m in range(len(seqch)):
        _,m = generalizedOpsName(seqch[n],seqch[m])
        mat.append(m)
    opstable.append(mat)

In [112]:
key = 'Cext'
f = open('palestrina.modal.'+key,'wb')
pickle.dump(rn,f)
pickle.dump(opstable,f)
f.close()

In [6]:
# Dictionary of modularity classes and relative keys
keydict = {0:'C', 1:'C', 2:'C', 3:'C', 4:'C', 5:'C',6:'C',7:'C'}
# keydict = {0:'G', 1:'G', 2:'G', 3:'G', 4:'G', 5:'G',}

In [7]:
# manual control of modulations
keychange = None #{20:'G',84:'G',98:'D',102:'C',154:'F',164:'D'}
# manual control of roman numerals
altrn = None #{64:'ii7[no5]',96:'vi7[no5]',147:'I[no3]',151:'iv7[no3]',155:'VII/o9[no3]'}

In [10]:
nxt,rn,ops = scoreAnalysis(seq,moduldict,keydict,first='ii[no3][no5]',
                           keychange=keychange,altrn=altrn,table='palestrina.modal.Cext',
                           verbose=True)

In [13]:
analysis = showAnalysis(nxt,chords,seq,rn,ops,keydict,moduldict,last=True,display=True)

In [12]:
analysis

,pcs,chord,rn,ops,key,modul
0,"[2, 7]",DG,v4,"O(0,0)",C,0
1,"[2, 7]",DG,v4,"O(0,0)",C,0
2,"[2, 7]",DG,v4,"O(0,0)",C,0
3,"[2, 7]",DG,v4,"O(0,0,-2)",C,0
4,"[2, 5, 7]",DFG,v7[no3],"O(2,-1,0)",C,0
...,...,...,...,...,...,...
365,"[5, 9, 11, 0]",FABC,IV764,"O(0,0,1,0)",C,5
366,"[5, 9, 0]",FAC,IV64,"O(2,0,0,-1,0)",C,5
367,"[4, 5, 7, 9, 0]",EFGAC,IV6543,"O(1,0,-2,0,0)",C,5
368,"[5, 9, 0]",FAC,IV64,"O(2,0,0,0)",C,5


In [ ]:
analysis.to_csv('analysis.csv')

In [ ]:
rnC[0] = 'C: '+ rnC[0]
rnG[0] = 'G: '+ rnG[0]
l = 0
analyzed = copy.deepcopy(chords)
for c in analyzed.recurse().getElementsByClass('Chord'):
    c.closedPosition(forceOctave=4,inPlace=True)
    c.addLyric('')
    c.addLyric(str(rnC[l]))
#     c.addLyric(str(ops[l]))
    c.addLyric(str(rnG[l]))
    l += 1
analyzed.show('musicxml')

In [122]:
seq[2],ops[2]

([2], 'O(0,5)')

In [13]:
df1 = pd.DataFrame({'lkey': ['foo', 'bar', 'baz', 'foo'],
                    'value': [1, 2, 3, 5]})
df2 = pd.DataFrame({'lkey': ['foo1', 'bar1', 'baz1', 'foo1'],
                    'value': [5, 6, 7, 8]})

In [21]:
df3 = df1.merge(df2,how='outer')

In [44]:
df3.drop_duplicates(subset='value')

,lkey,value
0,foo,1
1,bar,2
2,baz,3
3,foo,5
5,bar1,6
6,baz1,7
7,foo1,8
